In [ ]:
def build_model():
    import pandas as pd 
    from sklearn.linear_model import LinearRegression
    import joblib
    df = pd.read_csv('houses.csv')
    X = df[['size', 'nb_rooms', 'garden']]
    y = df['price']
    model = LinearRegression()
    model.fit(X, y)
    joblib.dump(model, "regression.joblib")
    
build_model()

In [ ]:
import joblib
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# Chargement du modèle de regression et récupération des coefficient
model = joblib.load("regression.joblib")

coef = model.coef_
intercept = model.intercept_

# On génère la chaîne de caractère contenant le code C et on l'ouvre
c_code = """
#include <stdio.h>

float prediction(float *features, int n_features) {{
    float coef[{n_coef}] = {{{coef_str}}};
    float intercept = {intercept};

    float result = 0.0;
    for (int i=0; i<n_features; i++) {{
        result += coef[i] * features[i];
    }}
    result += intercept;
    return result;
}}

int main() {{
    float features[] = {{1.0, 2.0, 3.0}}; // Exemple de données d'entrée
    int n_features = sizeof(features)/sizeof(float);
    float pred = prediction(features, n_features);
    printf("Prediction : %f\\n", pred);
    return 0;
}}
""".format(n_coef=len(coef), coef_str=", ".join(str(c) for c in coef), intercept=intercept)

with open("modele.c", "w") as f:
    f.write(c_code)

# Compilation du code C
compilation_command = "gcc modele.c -o modele"
print("Commande de compilation : {}".format(compilation_command))

Commande de compilation : gcc modele.c -o modele


In [ ]:
from sklearn.model_selection import train_test_split
# On charge ici les donnée depuis le fichier csv
data = pd.read_csv('houses.csv')
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

data['orientation'].replace({'Nord': 1, 'Sud': 2, 'Est': 3, 'Ouest': 4}, inplace=True)

X_train = data[['size', 'nb_rooms', 'garden', 'orientation']]
y_train = data['price']

model = LinearRegression()
model.fit(X_train, y_train)

#Modèle entrainé
joblib.dump(model, 'trained_model.joblib')

['trained_model.joblib']